## INFO834 TP2     


In [5]:
import pandas as pd

In [6]:

file_path_1 = './data/academies_virgule.csv'

file_path_2 = './data/villes_virgule.csv'

dfAcademies = pd.read_csv(file_path_1)
dfVilles = pd.read_csv(file_path_2)

print(dfAcademies)
print(dfVilles)


                         academie                              wikipedia  \
0                Académie de Lyon        fr:Académie de Lyon (éducation)   
1               Académie d'Amiens       fr:Académie d'Amiens (éducation)   
2    Académie de Clermont-Ferrand        fr:Académie de Clermont-Ferrand   
3        Académie d'Aix-Marseille            fr:Académie d'Aix-Marseille   
4        Académie d'Aix-Marseille            fr:Académie d'Aix-Marseille   
..                            ...                                    ...   
96         Académie de Versailles  fr:Académie de Versailles (éducation)   
97      Académie de la Guadeloupe           fr:Académie de la Guadeloupe   
98      Académie de la Martinique           fr:Académie de la Martinique   
99             Académie de Guyane               fr:Académie de la Guyane   
100        Académie de la Réunion              fr:Académie de La Réunion   

    vacances  dep              departement                       region  
0     Zone A 